In [2]:
!pip install -r requirements.txt

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
import numpy as np
import torch
import torchvision
from tqdm.notebook import tqdm
import os
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader
import albumentations as A
import cv2
from tqdm.auto import tqdm
from albumentations.pytorch import ToTensorV2
from torch.cuda.amp import GradScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [1]:
!tar -xf car_color_dataset.zip

# CrossVal

In [7]:
size = 256
def color2label(color):
    c2l = {
        'Black': 0,
        'Blue': 1,
        'Brown': 2,
        'Cyan': 3,
        'Green': 4,
        'Grey': 5,
        'Orange': 6,
        'Red': 7,
        'Violet': 8,
        'White': 9,
        'Yellow': 10
    }

    return c2l[color]
def label2color(label):
    c2l = {
        'Black': 0,
        'Blue': 1,
        'Brown': 2,
        'Cyan': 3,
        'Green': 4,
        'Grey': 5,
        'Orange': 6,
        'Red': 7,
        'Violet': 8,
        'White': 9,
        'Yellow': 10
    }
    l2c = {v: k for k, v in c2l.items()}
    return l2c[label]
def label_weight(label):
    lw = {
        0: 2,
        1: 3,
        2: 8,
        3: 3,
        4: 3,
        5: 2,
        6: 1,
        7: 1,
        8: 8,
        9: 2,
        10: 2
    }

    return lw[label]
class CarsDataset3(torch.utils.data.Dataset):
    def __init__(self, path_to_data, transform=False, train=False):
        self.path_to_data = path_to_data
        self.transform = transform
        self.train = train
        self.data = []
        self.labels = []
        self._get_data_from_path()
        self.data = np.array(self.data)
        self.labels = np.array(self.labels)

    def _get_data_from_path(self):
        for folder in tqdm(os.listdir(self.path_to_data)):
            img_folder = os.path.join(self.path_to_data, folder)
            for img in os.listdir(img_folder):
                img = cv2.imread(os.path.join(img_folder, img))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                label = color2label(folder)
                size = 256
                transform = A.Compose([
                                        A.LongestMaxSize(size),
                                        A.PadIfNeeded(size, size),
                                        A.CenterCrop(224, 224)])
                transformed = transform(image=img)
                img_tensor = transformed['image']
                self.data.append(img_tensor)
                self.labels.append(label)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
        
    def __len__(self):
        return self.data.shape[0]
    

In [5]:
path_to_train = './train'
train_dataset = CarsDataset3(path_to_train)

  0%|          | 0/11 [00:00<?, ?it/s]

In [8]:
import math
import torch
import torch.optim

from typing import TYPE_CHECKING, Any, Callable, Optional

if TYPE_CHECKING:
    from torch.optim.optimizer import _params_t
else:
    _params_t = Any

class MADGRAD(torch.optim.Optimizer):
    def __init__(
        self, params: _params_t, lr: float = 1e-2, momentum: float = 0.9, 
        weight_decay: float = 0, eps: float = 1e-6, decouple_decay=False,
    ):
        if momentum < 0 or momentum >= 1:
            raise ValueError(f"Momentum {momentum} must be in the range [0,1)")
        if lr <= 0:
            raise ValueError(f"Learning rate {lr} must be positive")
        if weight_decay < 0:
            raise ValueError(f"Weight decay {weight_decay} must be non-negative")
        if eps < 0:
            raise ValueError(f"Eps must be non-negative")

        defaults = dict(lr=lr, eps=eps, momentum=momentum, 
                        weight_decay=weight_decay, decouple_decay=decouple_decay)
        super().__init__(params, defaults)

    @property
    def supports_memory_efficient_fp16(self) -> bool:
        return False

    @property
    def supports_flat_params(self) -> bool:
        return True

    def step(self, closure: Optional[Callable[[], float]] = None) -> Optional[float]:
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        # step counter must be stored in state to ensure correct behavior under
        # optimizer sharding
        if 'k' not in self.state:
            self.state['k'] = torch.tensor([0], dtype=torch.long)
        k = self.state['k'].item()

        for group in self.param_groups:
            eps = group["eps"]
            lr = group["lr"] + eps
            decay = group["weight_decay"]
            momentum = group["momentum"]
            decouple_decay = group["decouple_decay"]

            ck = 1 - momentum
            lamb = lr * math.pow(k + 1, 0.5)

            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad.data
                state = self.state[p]

                if "grad_sum_sq" not in state:
                    state["grad_sum_sq"] = torch.zeros_like(p.data).detach()
                    state["s"] = torch.zeros_like(p.data).detach()
                    if momentum != 0:
                        state["x0"] = torch.clone(p.data).detach()

                if momentum != 0.0 and grad.is_sparse:
                    raise RuntimeError("momentum != 0 is not compatible with sparse gradients")

                grad_sum_sq = state["grad_sum_sq"]
                s = state["s"]

                # Apply weight decay
                if decay != 0 and not decouple_decay:
                    if grad.is_sparse:
                        raise RuntimeError("weight_decay option is not compatible with sparse gradients")

                    grad.add_(p.data, alpha=decay)

                if grad.is_sparse:
                    grad = grad.coalesce()
                    grad_val = grad._values()

                    p_masked = p.sparse_mask(grad)
                    grad_sum_sq_masked = grad_sum_sq.sparse_mask(grad)
                    s_masked = s.sparse_mask(grad)

                    # Compute x_0 from other known quantities
                    rms_masked_vals = grad_sum_sq_masked._values().pow(1 / 3).add_(eps)
                    x0_masked_vals = p_masked._values().addcdiv(s_masked._values(), rms_masked_vals, value=1)

                    # Dense + sparse op
                    grad_sq = grad * grad
                    grad_sum_sq.add_(grad_sq, alpha=lamb)
                    grad_sum_sq_masked.add_(grad_sq, alpha=lamb)

                    rms_masked_vals = grad_sum_sq_masked._values().pow_(1 / 3).add_(eps)

                    if eps == 0:
                        rms_masked_vals[rms_masked_vals == 0] = float('inf')

                    s.add_(grad, alpha=lamb)
                    s_masked._values().add_(grad_val, alpha=lamb)

                    # update masked copy of p
                    p_kp1_masked_vals = x0_masked_vals.addcdiv(s_masked._values(), rms_masked_vals, value=-1)
                    # Copy updated masked p to dense p using an add operation
                    p_masked._values().add_(p_kp1_masked_vals, alpha=-1)
                    p.data.add_(p_masked, alpha=-1)
                else:
                    if momentum == 0:
                        # Compute x_0 from other known quantities
                        rms = grad_sum_sq.pow(1 / 3).add_(eps)
                        x0 = p.data.addcdiv(s, rms, value=1)
                    else:
                        x0 = state["x0"]

                    # Accumulate second moments
                    grad_sum_sq.addcmul_(grad, grad, value=lamb)
                    rms = grad_sum_sq.pow(1 / 3).add_(eps)

                    if eps == 0:
                        rms[rms == 0] = float('inf')

                    # Update s
                    s.data.add_(grad, alpha=lamb)

                    if decay != 0 and decouple_decay:
                        p_old = p.data.clone()

                    # Step
                    if momentum == 0:
                        p.data.copy_(x0.addcdiv(s, rms, value=-1))
                    else:
                        z = x0.addcdiv(s, rms, value=-1)

                        # p is a moving average of z
                        p.data.mul_(1 - ck).add_(z, alpha=ck)
                    
                    if decay != 0 and decouple_decay:
                        p.data.add_(p_old, alpha=-lr*decay)


        self.state['k'] += 1
        return loss

In [9]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, scaler, device="cuda:0"):
    model.train()
    total_loss = 0
    total_accuracy = 0
    n_batchs = 0
    for X_batch, y_batch in train_dataloader:
        optimizer.zero_grad()
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            y_preds = model(X_batch)
            loss = criterion(y_preds, y_batch)
            total_loss += loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_accuracy += accuracy_score(y_batch.cpu().numpy(), torch.argmax(y_preds, dim=-1).detach().cpu().numpy())
        n_batchs += 1
    total_accuracy /= n_batchs
    total_loss /= n_batchs
    return total_accuracy, total_loss

def predict(model, val_dataloader, criterion, device="cuda:0"):
    model.eval()
    predicted_classes = torch.tensor([])
    true_classes = torch.tensor([])
    total_loss = 0
    n_batchs = 0
    with torch.no_grad():
        for X_batch, y_batch in val_dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_preds = model(X_batch)
            total_loss += criterion(y_preds, y_batch)
            y_batch = y_batch.cpu()
            predicts = model(X_batch)
            predicts = torch.argmax(predicts, dim=1).cpu()
            predicted_classes = torch.cat((predicted_classes, predicts), dim=-1)
            true_classes = torch.cat((true_classes, y_batch), dim=-1)
            n_batchs += 1
    total_loss /= n_batchs
    return total_loss, predicted_classes.numpy(), true_classes.numpy()

def train(model, train_dataloader, val_dataloader, criterion, optimizer, scaler, lr_scheduler, device="cuda:0", n_epochs=10):
    model.to(device)
    for epoch in tqdm(range(n_epochs)):
        train_acc, train_loss = train_one_epoch(model, train_dataloader, criterion, optimizer, scaler, device=device)
        if lr_scheduler is not None: 
            lr_scheduler.step()
        val_loss, predicted_classes, true_classes = predict(model, val_dataloader, criterion, device=device)
        val_acc = accuracy_score(true_classes, predicted_classes)
        print(f'Train Loss:{train_loss} Train Accuracy:{train_acc}')
        print(f'Val Loss:{val_loss} Val Accuracy:{val_acc} ')
        del true_classes, predicted_classes

In [10]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from torchvision.models import efficientnet_v2_l
import gc
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    #A.Blur(blur_limit=2.5, p=0.3),
    A.OneOf([
            A.CLAHE(clip_limit=2, p=0.75),
            A.RandomBrightnessContrast(p=0.75),
            A.ImageCompression(quality_lower=90, quality_upper=100, p=0.75),
        ], p=0.5),
    A.OneOf([
                A.Blur(blur_limit=2.5, p=0.75),
                A.MotionBlur(blur_limit=(3, 5), p=0.75),
            ], p=0.5),
    A.OneOf([
                A.Cutout(num_holes=12, max_h_size=21, max_w_size=21, p=0.75),
                A.Cutout(num_holes=6, max_h_size=42, max_w_size=42, p=0.75),
            ], p=0.5),
])
val_transform = A.Compose([
    A.Normalize(), 
    ToTensorV2()])
class cv_dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, transform, second_transform=None, train=False):
        self.data = data
        self.labels = labels
        self.transform  = transform
        self.second_transform = second_transform
        self.train = train
        self._add_augmentations()
        if self.train:
            #self._add_second_aug()
            data = self.data.copy()
            labels = self.labels.copy()
            del self.data, self.labels
            self.data = []
            self.labels = []
            for i in tqdm(range(len(data))):
                img = data[i]
                label = labels[i]
                transforms = A.Compose([
                                A.Normalize(), 
                                ToTensorV2()])
                transformed = transforms(image=img)
                img_tensor = transformed['image']
                self.data.append(img_tensor)
                self.labels.append(label)
        
    def _add_augmentations(self):
        data = self.data.copy()
        labels = self.labels.copy()
        del self.data, self.labels
        self.data = []
        self.labels = []
        for i in tqdm(range(len(data))):
            img = data[i]
            label = labels[i]
            weight =  1
            for j in range(weight):
                transformed = self.transform(image=img)
                img_tensor = transformed['image']
                self.data.append(img_tensor)
                self.labels.append(label)
                
    def _add_second_aug(self):
        data = self.data.copy()
        labels = self.labels.copy()
        for i in tqdm(range(len(data))):
            img = data[i]
            label = labels[i]
            transformed = self.second_transform(image=img)
            img_tensor = transformed['image']
            self.data.append(img_tensor)
            self.labels.append(label)
        del data, labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
def crossval(dataset, train_transform=train_transform, val_transform=val_transform,second_transform=None, n_folds=6):
    kf = KFold(n_splits=n_folds, random_state=42, shuffle=True)
    kf.get_n_splits(dataset.data)
    i = 0
    for train_index, test_index in kf.split(dataset.data, dataset.labels):
        X_train, y_train = dataset[train_index]
        X_test, y_test = dataset[test_index]
        fold_train = cv_dataset(X_train, y_train, train_transform, train=True)
        fold_val = cv_dataset(X_test, y_test, val_transform, train=False)
        train_dataloader = DataLoader(fold_train, batch_size=16, drop_last=True, shuffle=True)
        val_dataloader = DataLoader(fold_val , batch_size=16, drop_last=True, shuffle=True)
        device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
        model = efficientnet_v2_l(weights='IMAGENET1K_V1')
        model.classifier = nn.Linear(1280, 11)
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = MADGRAD(model.parameters(), 5e-5, weight_decay=4e-4, momentum=0.9)
        scaler = GradScaler()
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15, eta_min=3e-7, last_epoch=- 1, verbose=False)
        train(model, train_dataloader, val_dataloader, criterion, optimizer, scaler, lr_scheduler, device="cuda:0", n_epochs=15)
        with open(f'./models/model_{i}.pkl', 'wb') as f:
            pickle.dump(model, f)
        del model, train_dataloader, val_dataloader, fold_train, fold_val
        gc.collect()
        torch.cuda.empty_cache()
        i += 1

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


In [41]:
crossval(train_dataset, train_transform=train_transform, val_transform=val_transform, n_folds=5)

100%|██████████| 761/761 [00:00<00:00, 4774.65it/s]
Downloading: "https://download.pytorch.org/models/efficientnet_v2_l-59c71312.pth" to /home/jovyan/.cache/torch/hub/checkpoints/efficientnet_v2_l-59c71312.pth


  0%|          | 0.00/455M [00:00<?, ?B/s]

  7%|▋         | 1/15 [01:13<17:08, 73.47s/it]

Train Loss:0.662440836429596 Train Accuracy:0.8075657894736842
Val Loss:0.15683378279209137 Val Accuracy:0.9601063829787234 


 13%|█▎        | 2/15 [02:26<15:54, 73.45s/it]

...

Train Loss:0.003609290812164545 Train Accuracy:1.0
Val Loss:0.11569105088710785 Val Accuracy:0.9707446808510638 


100%|██████████| 15/15 [18:15<00:00, 73.02s/it]

Train Loss:0.0020041191019117832 Train Accuracy:1.0
Val Loss:0.11032257229089737 Val Accuracy:0.9667553191489362 



  0%|          | 0/15 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.68 GiB total capacity; 6.95 GiB already allocated; 28.56 MiB free; 7.00 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [14]:
import gc
gc.collect()

0

In [15]:
torch.cuda.empty_cache()